# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iskateley,67.6803,53.1512,-30.55,100,95,1.79,RU,1707628472
1,1,port-aux-francais,-49.3500,70.2167,7.95,99,100,12.31,TF,1707628472
2,2,grytviken,-54.2811,-36.5092,2.85,89,28,7.70,GS,1707628472
3,3,itanhem,-17.1664,-40.3300,20.70,94,76,1.09,BR,1707628473
4,4,college,64.8569,-147.8028,-7.82,78,40,2.57,US,1707628473


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s

reduced_df = city_data_df[(city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) & (city_data_df['Wind Speed'] <= 4.5)]

# Drop any rows with null values
reduced_df.dropna(how='any')

# Display sample data
reduced_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,tutoia,-2.7619,-42.2744,23.64,96,100,2.11,BR,1707628473
9,9,hawaiian paradise park,19.5933,-154.9731,21.73,91,100,1.54,US,1707628474
21,21,teshi old town,5.5836,-0.1072,26.87,86,21,2.94,GH,1707628477
28,28,blackmans bay,-43.0167,147.3167,22.07,52,73,3.56,AU,1707628479
39,39,kapa'a,22.0752,-159.3190,22.99,83,100,2.57,US,1707628482


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/bs/50jvlwm13zs8zkw2m897gk9w0000gn/T/ipykernel_11789/822424206.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
6,tutoia,BR,-2.7619,-42.2744,96,
9,hawaiian paradise park,US,19.5933,-154.9731,91,
21,teshi old town,GH,5.5836,-0.1072,86,
28,blackmans bay,AU,-43.0167,147.3167,52,
39,kapa'a,US,22.0752,-159.3190,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tutoia - nearest hotel: Pousada San Vicente
hawaiian paradise park - nearest hotel: No hotel found
teshi old town - nearest hotel: Ingot Hotel
blackmans bay - nearest hotel: Villa Howden
kapa'a - nearest hotel: Pono Kai Resort
sao vicente - nearest hotel: Pousada Vitória
turan - nearest hotel: Thanh Thanh Hotel
hadibu - nearest hotel: No hotel found
sittwe - nearest hotel: Yuzana Aung Motel 1
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
hilo - nearest hotel: Dolphin Bay Hotel
valera - nearest hotel: Valera
talodi - nearest hotel: No hotel found
ankazoabo - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
howrah - nearest hotel: Sun India Guest House
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
malango - nearest hotel: Madarana
namibe - nearest hotel: Hotel Chik Chik Namibe
canutama - nearest hotel: No hotel found
luebo - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
6,tutoia,BR,-2.7619,-42.2744,96,Pousada San Vicente
9,hawaiian paradise park,US,19.5933,-154.9731,91,No hotel found
21,teshi old town,GH,5.5836,-0.1072,86,Ingot Hotel
28,blackmans bay,AU,-43.0167,147.3167,52,Villa Howden
39,kapa'a,US,22.0752,-159.3190,83,Pono Kai Resort
...,...,...,...,...,...,...
539,beneditinos,BR,-5.4500,-42.3667,99,No hotel found
540,puerto baquerizo moreno,EC,-0.9000,-89.6000,86,Dory's House
543,opuwo,NaN,-18.0607,13.8400,78,Ohakane Lodge
546,aripuana,BR,-9.1667,-60.6333,99,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [37]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)